In [1]:
import pandas as pd
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
# we import the training set 
train=pd.read_excel('trainReputationLowNoise.xlsx' )
train=train.iloc[:,:-2].astype(float)

train

,PantsTotal,NotRealTotal,BarelyTotal,HalfTotal,MostlyTotal,Truths
0,1.0,0.0,1.0,1.0,0.0,3.0
1,0.0,2.0,1.0,1.0,0.0,2.0
2,1.0,0.0,1.0,1.0,0.0,3.0
3,0.0,1.0,1.0,1.0,2.0,1.0
4,1.0,1.0,2.0,0.0,1.0,1.0
...,...,...,...,...,...,...
6529,9.0,71.0,70.0,160.0,163.0,125.0
6530,9.0,71.0,70.0,160.0,163.0,125.0
6531,9.0,71.0,70.0,160.0,163.0,125.0
6532,9.0,71.0,70.0,160.0,163.0,125.0


In [3]:
#we import the one hot encoded labels
TrainLables=pd.read_excel('trainReputationLowNoise.xlsx' )
TrainLables=TrainLables.iloc[:,-1] 

TrainLables=pd.get_dummies(TrainLables)
TrainLables

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,1,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
6529,0,0,0,0,1,0
6530,0,0,0,0,1,0
6531,0,0,0,1,0,0
6532,0,0,0,0,0,1


In [4]:
#get input vector from dataset in this cell
 
input=torch.tensor(train.values)
input

tensor([[  1.,   0.,   1.,   1.,   0.,   3.],
        [  0.,   2.,   1.,   1.,   0.,   2.],
        [  1.,   0.,   1.,   1.,   0.,   3.],
        ...,
        [  9.,  71.,  70., 160., 163., 125.],
        [  9.,  71.,  70., 160., 163., 125.],
        [  9.,  71.,  70., 160., 163., 125.]], dtype=torch.float64)

In [5]:
#we get the size of the output of our neural network from classification

 
targets=torch.tensor(TrainLables.astype(float).values)

targets

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.]], dtype=torch.float64)

In [6]:
 
size= torch.tensor(input[0].size())
InputSize=size.item()

OutputSize=torch.tensor(targets[0].size()).item()

print('input size:', InputSize)
print('output size:', OutputSize)

input size: 6
output size: 6


In [7]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
         
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(InputSize,24)   
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, OutputSize)  #classifies 'outputsize' different classes

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)) 
        x = torch.sigmoid(self.fc3(x)).double()
        return x

    

#now we use it

net =Net()

In [8]:
# here we  setup the neural network parameters
# pick an optimizer (Simple Gradient Descent)

learning_rate = 9e-4

criterion = nn.MSELoss()  #computes the loss Function

import torch.optim as optim

# creating optimizer
#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [9]:
 # this is how to compute the loss, backporpagate and update the weights

# in the training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input.float())

loss = criterion(output, targets)
print('Loss:', loss, end=' ')

loss.backward()  #backprop
optimizer.step()    # Does the update

Loss: tensor(0.2651, dtype=torch.float64, grad_fn=<MseLossBackward>) 

In [53]:
# next step is to apply this in epochs 
for epoch in range(1000):  

        
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input.float())

    loss = criterion(output, targets)
    print('Loss:', loss, ' at epoch:', epoch)

    loss.backward()  #backprop
    optimizer.step()    # Does the update

Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 0
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 1
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 3
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 4
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 5
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 6
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 8
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 9
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 10
Loss: tensor(0.1284, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 11
Loss: tensor(0

Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 100
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 101
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 102
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 103
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 104
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 105
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 106
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 107
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 108
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 109
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 110
Loss: tensor(0.1283, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 205
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 206
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 207
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 208
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 209
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 210
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 211
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 212
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 213
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 214
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 215
Loss: tensor(0.1282, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 310
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 311
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 312
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 313
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 314
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 315
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 316
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 317
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 318
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 319
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 320
Loss: tensor(0.1281, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 413
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 414
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 415
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 416
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 417
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 418
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 419
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 420
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 421
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 422
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 423
Loss: tensor(0.1280, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 529
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 530
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 531
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 532
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 533
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 534
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 535
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 536
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 537
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 538
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 539
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 629
Loss: tensor(0.1279, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 630
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 631
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 632
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 633
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 634
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 635
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 636
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 637
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 638
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 639
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 735
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 736
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 737
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 738
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 739
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 740
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 741
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 742
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 743
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 744
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 745
Loss: tensor(0.1278, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 846
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 847
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 848
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 849
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 850
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 851
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 852
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 853
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 854
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 855
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 856
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 959
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 960
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 961
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 962
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 963
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 964
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 965
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 966
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 967
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 968
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 969
Loss: tensor(0.1277, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

In [54]:
correct = 0
total = 0

predicted=0  #how many wer predicted correctly
 

with torch.no_grad():
    for row in range(len(train)):
        outputs = net(input[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if TrainLables.iloc[row,result]==1: correct+=1
        
        print(result, end=' ')
    
 
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ', ( 100 * correct / total), '%')

5 1 5 4 2 4 1 4 5 3 3 3 4 3 1 4 4 4 2 2 2 1 4 4 4 2 4 4 2 4 2 2 2 1 2 5 4 1 1 1 1 4 4 1 1 1 1 3 4 3 4 4 1 3 2 1 4 1 4 4 4 4 4 4 1 4 1 4 3 1 4 1 1 4 1 1 1 4 1 2 4 4 1 3 3 5 2 1 2 1 1 4 1 1 1 4 4 4 1 5 2 1 4 3 4 2 4 1 1 4 4 5 1 1 3 4 1 4 4 4 4 4 2 1 0 0 0 0 0 0 1 1 3 4 5 3 3 2 5 1 5 5 5 1 5 5 5 1 4 4 4 4 3 4 4 2 4 3 4 3 2 3 4 4 4 4 4 3 5 4 4 5 4 4 1 4 3 1 1 3 1 4 4 3 4 4 4 4 3 5 5 5 1 4 2 1 3 4 2 5 3 5 3 3 5 5 5 1 4 5 4 4 3 3 3 4 4 4 3 4 3 3 4 3 5 2 3 5 4 1 1 4 3 2 4 4 3 4 3 4 1 4 2 4 2 4 4 4 4 4 4 4 1 4 2 2 5 5 4 5 3 3 3 5 5 3 4 3 3 3 1 3 3 5 5 3 4 4 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 5 4 3 3 1 1 2 2 2 3 4 1 5 4 2 4 4 5 1 4 3 4 3 4 3 4 5 1 2 2 1 5 4 4 2 4 1 5 4 5 5 1 1 5 4 1 1 1 5 5 4 4 3 5 4 1 2 5 5 5 1 5 4 5 3 4 5 3 4 5 1 4 5 2 4 4 3 3 2 2 4 3 4 5 5 3 5 3 5 3 5 4 3 5 4 4 4 5 5 4 5 5 4 1 4 4 5 3 5 5 4 3 3 1 1 4 4 4 4 2 2 5 2 4 4 4 4 4 5 4 5 5 3 3 3 3 1 4 1 1 1 4 4 4 2 3 3 4 3 3 3 5 4 1 4 3 4 4 3 3 3 4 3 4 4 3 5 3 4 4 4 4 3 3 3 4 3 3 5 5 5 3 3 1 4 4 1 4 1 1 1 4 5 4 3 4 3 5 4 4 3 4 5 1 

 3 0 3 3 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 0 0 3 0 0 0 3 0 3 3 3 3 0 0 3 3 3 3 3 0 3 3 3 3 0 3 0 3 0 0 3 3 3 3 0 0 0 3 0 3 3 0 0 3 0 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3

In [55]:
EvalData=pd.read_excel('validReputation.xlsx' )
EvalData=EvalData.iloc[:,:-2].astype(float)

EvalData=torch.tensor(EvalData.values)
EvalData

tensor([[  4.,  10.,  11.,  21.,  16.,  13.],
        [  2.,   6.,   6.,  11.,   5.,   4.],
        [  9.,  71.,  70., 160., 163., 125.],
        ...,
        [  0.,   0.,   0.,   0.,   1.,   0.],
        [  0.,  10.,   8.,  12.,   5.,   4.],
        [  0.,   0.,   0.,   1.,   1.,   0.]], dtype=torch.float64)

In [56]:
EvalLables=pd.read_excel('validReputation.xlsx' )
EvalLables=EvalLables.iloc[:,-1]
EvalLables=pd.get_dummies(EvalLables)
EvalLables=EvalLables.astype(float)
#EvalLables=torch.tensor(EvalLables.values) 

EvalLables


,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
1279,0.0,1.0,0.0,0.0,0.0,0.0
1280,0.0,0.0,1.0,0.0,0.0,0.0
1281,0.0,0.0,0.0,0.0,1.0,0.0
1282,0.0,0.0,0.0,0.0,1.0,0.0


In [57]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(EvalData)):
        outputs = net(EvalData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if EvalLables.iloc[row,result]==1: correct+=1
            
        
        Y.append(result)
        Pred.append(EvalLables.iloc[row])
        
        print(result, end=' ')
        
       
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ', ( 100 * correct / total), '%')

3 3 4 5 4 3 5 3 1 4 3 2 1 2 3 4 4 2 0 3 4 3 1 3 4 4 4 4 1 4 1 4 4 1 0 0 4 4 4 0 0 1 2 0 1 3 1 4 4 4 4 1 0 4 1 1 4 1 4 5 3 1 1 1 1 3 0 1 1 4 4 1 1 0 1 4 1 4 4 1 1 2 4 3 4 4 3 4 5 1 1 2 1 3 4 1 3 4 0 1 0 3 1 1 5 4 5 0 4 1 1 1 4 3 1 1 1 2 4 1 5 4 5 0 3 4 4 4 5 3 3 3 2 2 1 2 3 2 3 1 3 4 4 5 3 3 1 3 3 4 4 4 5 1 4 2 5 2 3 2 3 5 4 3 5 4 4 5 1 3 1 1 3 1 1 4 5 4 2 3 1 1 4 3 3 5 4 5 3 3 0 4 3 4 4 3 2 5 4 1 3 4 3 3 3 2 4 4 3 2 1 1 1 1 1 3 3 4 4 3 4 4 1 3 0 1 2 1 2 4 3 3 2 2 1 2 1 1 2 0 3 5 2 3 3 4 3 4 2 1 3 3 1 1 2 2 4 4 3 5 5 5 3 1 5 3 5 3 3 1 2 1 4 4 3 1 2 1 5 5 1 1 5 4 3 1 4 0 3 1 1 1 1 4 4 5 1 1 3 4 4 3 2 5 3 4 5 3 3 1 1 3 4 2 2 3 5 1 1 1 5 3 1 3 1 1 4 4 3 4 5 2 3 5 4 2 4 1 1 5 1 1 1 1 4 0 2 3 3 3 1 4 3 5 2 2 4 1 2 2 2 4 4 2 4 4 4 3 2 1 4 1 3 5 2 1 5 5 2 1 4 1 3 2 5 3 2 3 5 3 1 1 5 0 4 3 5 1 1 3 4 3 3 5 5 5 4 3 4 3 3 1 4 3 3 4 5 4 4 1 4 2 4 4 1 3 2 1 3 5 3 0 2 3 4 1 0 5 5 1 4 3 4 4 4 3 1 3 3 3 5 2 3 3 5 3 1 3 3 1 3 2 3 3 1 4 5 3 5 3 3 1 3 1 4 5 5 1 4 4 5 0 1 3 3 3 3 3 1 3 3 3 3 3 3 2 3 3 3 3 

In [46]:
#load previously saved model 

#PATH = './Sigmoid_linearMSE_adam_43.pth'
#net = Net()
#net.load_state_dict(torch.load(PATH))

In [58]:
# load the test data

TestData=pd.read_excel('testReputation.xlsx' )
TestData=TestData.iloc[:,:-2].astype(float)
TestData=torch.tensor(TestData.values)
TestData

tensor([[ 1.,  2.,  1.,  3.,  8.,  2.],
        [ 0.,  0.,  0.,  3.,  3.,  2.],
        [ 4.,  5., 12.,  9.,  3.,  8.],
        ...,
        [ 2.,  1.,  5.,  6.,  1.,  1.],
        [44., 19.,  7.,  3.,  5.,  3.],
        [ 1., 12.,  2.,  8.,  3.,  1.]], dtype=torch.float64)

In [59]:
labels=pd.read_excel('testReputation.xlsx' )

labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
1278,0,1,0,0,0,0
1279,0,0,0,0,1,0
1280,0,0,1,0,0,0
1281,1,0,0,0,0,0


In [60]:
TestLables =torch.tensor(labelsOneHot.values)
TestLables

tensor([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [61]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(TestData)):
        outputs = net(TestData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(result)
        Pred.append(labels.iloc[row])
        
        print(result, end=' ')
        
       
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ', ( 100 * correct / total), '%')

4 4 2 1 3 3 4 4 5 4 4 3 3 5 3 3 1 3 3 4 4 4 3 1 1 4 1 4 3 4 4 3 0 4 4 1 4 4 4 4 1 1 4 1 4 4 3 4 4 4 4 1 1 3 1 4 2 2 4 1 1 1 4 3 1 4 3 3 5 4 3 5 1 1 1 1 1 1 4 4 1 1 1 4 1 1 4 4 1 4 4 4 3 1 2 4 4 1 3 1 4 0 4 3 3 1 1 3 1 5 1 3 3 3 1 3 0 4 1 2 3 4 4 3 1 4 1 4 3 3 3 3 1 1 1 0 0 1 2 3 4 3 5 3 3 4 4 3 4 5 0 3 1 3 5 3 4 5 3 3 3 1 4 4 1 5 4 3 3 1 5 3 5 5 3 3 3 4 0 2 1 4 4 1 1 3 1 1 1 3 5 5 5 0 1 3 2 3 3 1 1 4 5 3 5 3 3 5 1 3 3 3 3 3 4 1 3 3 3 1 1 1 4 1 1 5 3 2 3 0 4 1 0 1 1 4 4 0 4 2 2 1 2 4 4 1 2 1 3 3 0 4 4 1 3 3 5 0 4 1 3 1 1 2 0 3 1 3 4 0 3 4 1 2 3 2 5 1 2 5 2 1 5 4 5 5 3 1 5 0 3 4 1 4 3 1 4 5 1 3 1 5 5 3 5 3 1 5 3 1 1 3 1 3 0 5 5 3 3 3 4 1 4 4 3 0 5 1 5 4 1 4 3 3 0 1 1 3 3 2 4 2 3 4 3 5 2 4 5 3 3 0 4 1 4 1 1 3 1 2 1 1 1 1 2 0 3 4 2 1 0 1 5 5 4 3 3 5 0 1 3 1 3 4 5 3 3 3 3 5 5 2 5 1 2 1 3 3 3 4 4 3 0 1 4 5 2 4 1 3 4 2 4 3 2 1 1 4 3 2 3 4 4 1 3 3 4 5 4 3 5 1 1 5 2 4 4 5 4 1 5 5 0 1 3 1 3 1 1 3 1 3 1 1 4 3 4 3 1 1 5 1 1 4 4 3 1 3 1 3 3 4 1 0 3 3 3 1 3 3 2 3 3 3 2 1 2 4 3 1 1 5 4 3 2 3 3 4 3 4 

In [62]:
from sklearn import metrics
 
print(metrics.confusion_matrix(Y,Pred))

[[ 53  14  11   1   3   3]
 [ 19 135  41  43  31  34]
 [  4  21  66  13  12  12]
 [ 11  36  56 119  51  29]
 [  2  29  31  61 138  62]
 [  3  15   9  30  14  71]]


In [34]:
target_names = ['Pants', 'False', 'Barely-True','Hlaf-True','Mostly-True','True']

print(metrics.classification_report(Y, Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.60      0.59      0.59        93
       False       0.53      0.43      0.48       308
 Barely-True       0.31      0.49      0.38       135
   Hlaf-True       0.41      0.39      0.40       281
 Mostly-True       0.51      0.42      0.46       298
        True       0.37      0.46      0.41       168

    accuracy                           0.44      1283
   macro avg       0.45      0.47      0.45      1283
weighted avg       0.46      0.44      0.45      1283



In [63]:
#save the model

PATH = './Sigmoid_linearMSE_adam_4536.pth'
torch.save(net.state_dict(), PATH)

# more on saving pytorch networks: https://pytorch.org/docs/stable/notes/serialization.html